In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs

In [3]:
def find_text(article,text_id):
    text_div = article.find('div', {'id': text_id})
    result = []
    if text_div is not None:
        for element in text_div.find_all(['span', 'img']):
            if element.name == 'span':
                result.append(element.get_text(strip=True))
            elif element.name == 'img':
                result.append(element.get('title', ''))
    else :
        return {"text":"NA"}    
    output_string = ' '.join(result)

    cleaned_text = ' '.join( output_string.split())
    return {"text":cleaned_text}
    

In [4]:
def first_value(x):
    y=x.split()
    return int(y[0])

In [5]:
def second_value(x):
    y=x.split()
    return y[1]

In [6]:
def find_dashboard(article,info_id):
    info_div = article.find('div', {'id': info_id})
    aria_label = info_div['aria-label'].split(",")
    data = {"replies":0,"reposts":0,"likes":0,"bookmarks": 0,"views":0}
    L=['reposts','likes','bookmarks','views','replies']
    for entry in aria_label:
        if len(entry.split()) == 2 and entry.split()[1] in L : 
            x_1 = first_value(entry)
            x_2 = second_value(entry)
            data[x_2] = x_1
    return data

In [7]:
def time_date(article,info_id):
    info_div = article.find('div', {'id': info_id})
    time_el = info_div.find('time')
    aria_label = time_el['datetime'].split("T")
    date = aria_label[0]
    time = aria_label[1]
    return {"date":date,"time":time[:8]}

In [8]:
def is_reposted(article,post_id):
    if len(article.find_all('a', {'id': post_id}))==0:
        return {"Is_reposted":0}
    else:
        return {"Is_reposted":1}

In [9]:
def photo_video_count(article,id):
    info_div = article.find('div', {'id': id})
    imgs = len(article.find_all('img', {'class':"css-9pa8cd"}))
    vids = len(article.find_all('video'))

    return {"Images":imgs-1,"Videos":vids}

In [10]:
import pandas as pd
df_data = pd.DataFrame()

In [11]:
username = input("Enter Valid Username")

Enter Valid Username jivrajS10694770


In [12]:
try:
    # Initialize the WebDriver
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    
    # Navigate to the login page
    driver.get("https://twitter.com/login")
    time.sleep(30)
    
    # Navigate to the user page
    driver.get("https://twitter.com/"+ username)
    # driver.maximize_window()
    time.sleep(5)
    
    # Set a delay between scroll actions (adjust as needed)
    scroll_delay = 0.3
    
    tracker = [] 
    id = 1
    count_50 = 50
    
    # Scroll down the page gradually
    for i in range(1000):   # Adjust the number of iterations as needed
        # Perform scroll action
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
    
        # Wait for a short delay to simulate slow loading
        time.sleep(scroll_delay)
    
        # Get the HTML source code of the current page
        html_source = driver.page_source
    
        soup_data = bs(html_source, 'html.parser')
    
        articles = soup_data.find_all('article')
    
        for article in articles:
            aria_labelledby_values = article['aria-labelledby'].split()
            if aria_labelledby_values[10] not in tracker:
                tracker.append(aria_labelledby_values[10])
                text = find_text(article, aria_labelledby_values[10])      # fixed
                dash = find_dashboard(article, aria_labelledby_values[-1])  # fixed
                repo = is_reposted(article, aria_labelledby_values[3])
                counts = photo_video_count(article, aria_labelledby_values[12])
                date_time = time_date(article,aria_labelledby_values[4])
                # Use the .append() method to add a new row to the DataFrame
                frames = [df_data, pd.DataFrame({'id': "is_" + str(id), **dash, **repo, **counts, **date_time, **text}, index=[0])]
                df_data = pd.concat(frames, ignore_index=True)
        
                id += 1
    
                if len(tracker) > 50 :
                    tracker.pop(0)
except:
    print("Something went wrong")


finally:
    # Close the WebDriver
    driver.quit()


In [13]:
df_data.shape

(8, 12)

In [14]:
data = df_data.set_index("id")

In [15]:
data

,replies,reposts,likes,bookmarks,views,Is_reposted,Images,Videos,date,time,text
id,,,,,,,,,,,
is_1,0,0,0,2,40,0,0,0,2023-12-27,07:16:17,We are at the end of this year. Bye bye 2023.
is_2,188,109,761,24,16103,1,1,1,2023-12-26,14:06:59,"""Am I an avatar in someone's video game?"" - @e..."
is_3,1711,4391,30498,551,3735505,1,4,0,2023-12-26,01:29:58,The Falcon fleet’s life leading rocket complet...
is_4,150,119,1256,22,76166,1,4,0,2023-12-26,17:29:54,What a difference a year makes Sparkles Did yo...
is_5,698,13274,128312,2394,10194302,1,1,1,2023-12-26,09:19:33,No one can replace our mom .... https:// jgbgq...
is_6,11260,9306,71940,269,2285340,1,1,0,2023-12-25,13:01:36,"From our family to yours: Merry Christmas, Ame..."
is_7,4400,7100,80706,897,4969442,1,1,0,2023-12-23,23:50:51,I Red heart Japan
is_8,629,518,5578,35,138513,1,1,0,2023-12-25,07:11:16,The Holiday season is the BEST season. Nothing...


In [16]:
df_data.to_csv('twitter_data.csv', index=False)